1. Load MERFISH output and export as spatialdata object.  
Two approaches: sdata containing transcript table plus
    1. images from z=3 plane
    2. images from all z planes

2. Take boundaries and adata files from each segmentation approach and add to main sdata

In [1]:
import json
import os

import spatialdata as sd

/dss/dsshome1/0C/ra98gaq/miniconda3/envs/sdata/lib/python3.11/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(


In [2]:
import glob
import os
import re

import spatialdata as sd
import spatialdata_io

/dss/dsshome1/0C/ra98gaq/miniconda3/envs/sdata/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  warnings.warn(msg, FutureWarning)


In [3]:
import sys

sys.path.append("/home")
import src.sdata_utils as sdata_utils

import importlib
importlib.reload(sdata_utils)

<module 'src.sdata_utils' from '/home/src/sdata_utils.py'>

In [4]:
data_dir = os.path.abspath("/home/data")
!ls -l $data_dir

lrwxrwxrwx 1 ra98gaq pn52ra 53 Mar 12 17:01 /home/data -> /dss/dssfs03/pn52re/pn52re-dss-0001/cellseg-benchmark


In [5]:
archive_dir = os.path.abspath("/home/archive")
!ls -l $archive_dir

lrwxrwxrwx 1 ra98gaq pn52ra 35 Mar 12 17:02 /home/archive -> /dss/dssfs03/pn52re/pn52re-dss-0000


In [6]:
with open(os.path.join(data_dir, "sample_paths.json")) as file:
    sample_paths = json.load(file)

# Create main sdata (transcripts and images only)

In [7]:
#!rm -r "{data_dir}/samples/foxf2_s2_r1/sdata_z3.zarr"

In [15]:
def process_merscope(sample_name, sdata_file, sample_paths, zmode):
    """Load and save a MERSCOPE sample as sdata with specified z_layers configuration. Only loads transcripts and mosaic_images."""

    if zmode not in {"z3", "3d"}:
        raise ValueError(f"Invalid zmode: {zmode}")
    if os.path.exists(sdata_file):
        print(f"Skipping {sample_name}: {zmode} file already exists")
        return
    sdata = spatialdata_io.merscope(
        sample_paths[sample_name],
        z_layers=3 if zmode == "z3" else range(7),
        backend=None,
        cells_boundaries=False,
        cells_table=False,
        mosaic_images=False,
        transcripts=True,
        slide_name="_".join(sample_name.split("_")[:2]),
        region_name=sample_name.split("_")[2],
    )
    os.makedirs(os.path.dirname(sdata_file), exist_ok=True)
    sdata.write(sdata_file, overwrite=False)

In [ ]:
# sample foxf2_s2_r1 (z3 and 3d)
sample_name = "foxf2_s2_r1"
for zmode in ["z3", "3d"]:
    sdata_utils.process_merscope(sample_name, 
                                 sdata_file = os.path.join(data_dir, "samples", sample_name, f"sdata_{zmode}_temp_SF.zarr"),
                                 sample_paths = sample_paths, 
                                 zmode = zmode
                                )

In [19]:
# all samples (z3 only)
zmodes = ["z3"]
for sample_name in sample_paths.keys():
    print("/nProcessing "+sample_name)
    for zmode in zmodes:
        process_merscope(sample_name, 
                         sdata_file = os.path.join(data_dir, "samples", sample_name, f"sdata_{zmode}_temp_SF.zarr"),
                         sample_paths = sample_paths, 
                         zmode = zmode
                        )

Processing foxf2_s1_r0
INFO     The column "global_x" has now been renamed to "x"; the column "x" was already present in the dataframe,   
         and will be dropped.                                                                                      
INFO     The column "global_y" has now been renamed to "y"; the column "y" was already present in the dataframe,   
         and will be dropped.                                                                                      


/dss/dsshome1/0C/ra98gaq/miniconda3/envs/sdata/lib/python3.11/functools.py:946: UserWarning: The index of the dataframe is not monotonic increasing. It is recommended to sort the data to adjust the order of the index before calling .parse() (or call `parse(sort=True)`) to avoid possible problems due to unknown divisions.
  return method.__get__(obj, cls)(*args, **kwargs)


INFO     The Zarr backing store has been changed from None the new file path:                                      
         /home/data/samples/foxf2_s1_r0/sdata_z3_temp_SF.zarr                                                      
Processing foxf2_s1_r1
INFO     The column "global_x" has now been renamed to "x"; the column "x" was already present in the dataframe,   
         and will be dropped.                                                                                      
INFO     The column "global_y" has now been renamed to "y"; the column "y" was already present in the dataframe,   
         and will be dropped.                                                                                      


/dss/dsshome1/0C/ra98gaq/miniconda3/envs/sdata/lib/python3.11/functools.py:946: UserWarning: The index of the dataframe is not monotonic increasing. It is recommended to sort the data to adjust the order of the index before calling .parse() (or call `parse(sort=True)`) to avoid possible problems due to unknown divisions.
  return method.__get__(obj, cls)(*args, **kwargs)


INFO     The Zarr backing store has been changed from None the new file path:                                      
         /home/data/samples/foxf2_s1_r1/sdata_z3_temp_SF.zarr                                                      
Processing foxf2_s2_r1
INFO     The column "global_x" has now been renamed to "x"; the column "x" was already present in the dataframe,   
         and will be dropped.                                                                                      
INFO     The column "global_y" has now been renamed to "y"; the column "y" was already present in the dataframe,   
         and will be dropped.                                                                                      


...

INFO     The Zarr backing store has been changed from None the new file path:                                      
         /home/data/samples/foxf2_s6_r2/sdata_z3_temp_SF.zarr                                                      
Processing foxf2_s7_r0
INFO     The column "global_x" has now been renamed to "x"; the column "x" was already present in the dataframe,   
         and will be dropped.                                                                                      
INFO     The column "global_y" has now been renamed to "y"; the column "y" was already present in the dataframe,   
         and will be dropped.                                                                                      


/dss/dsshome1/0C/ra98gaq/miniconda3/envs/sdata/lib/python3.11/functools.py:946: UserWarning: The index of the dataframe is not monotonic increasing. It is recommended to sort the data to adjust the order of the index before calling .parse() (or call `parse(sort=True)`) to avoid possible problems due to unknown divisions.
  return method.__get__(obj, cls)(*args, **kwargs)


INFO     The Zarr backing store has been changed from None the new file path:                                      
         /home/data/samples/foxf2_s7_r0/sdata_z3_temp_SF.zarr                                                      
Processing foxf2_s7_r1
INFO     The column "global_x" has now been renamed to "x"; the column "x" was already present in the dataframe,   
         and will be dropped.                                                                                      
INFO     The column "global_y" has now been renamed to "y"; the column "y" was already present in the dataframe,   
         and will be dropped.                                                                                      


/dss/dsshome1/0C/ra98gaq/miniconda3/envs/sdata/lib/python3.11/functools.py:946: UserWarning: The index of the dataframe is not monotonic increasing. It is recommended to sort the data to adjust the order of the index before calling .parse() (or call `parse(sort=True)`) to avoid possible problems due to unknown divisions.
  return method.__get__(obj, cls)(*args, **kwargs)


INFO     The Zarr backing store has been changed from None the new file path:                                      
         /home/data/samples/foxf2_s7_r1/sdata_z3_temp_SF.zarr                                                      


# Add data from segmentation outputs: shapes and adatas

In [7]:
sample_name = "foxf2_s2_r1"

In [8]:
# load main sdata
sdata_path = os.path.join(data_dir, "samples", sample_name, "sdata_z3_temp_SF.zarr")
if "sdata_main" not in locals():
    sdata_main = sd.read_zarr(sdata_path)

/dss/dsshome1/0C/ra98gaq/miniconda3/envs/sdata/lib/python3.11/site-packages/spatialdata/_core/spatialdata.py:185: UserWarning: The table is annotating 'baysor_boundaries', which is not present in the SpatialData object.
  self.validate_table_in_spatialdata(v)
/dss/dsshome1/0C/ra98gaq/miniconda3/envs/sdata/lib/python3.11/site-packages/spatialdata/_core/spatialdata.py:185: UserWarning: The table is annotating 'cellpose_boundaries', which is not present in the SpatialData object.
  self.validate_table_in_spatialdata(v)
/dss/dsshome1/0C/ra98gaq/miniconda3/envs/sdata/lib/python3.11/site-packages/spatialdata/_core/spatialdata.py:185: UserWarning: The table is annotating '20240322_Foxf2-Slide02-cp-WT-PCKO_region_1-WT536_polygons', which is not present in the SpatialData object.
  self.validate_table_in_spatialdata(v)
/dss/dsshome1/0C/ra98gaq/miniconda3/envs/sdata/lib/python3.11/site-packages/spatialdata/_core/spatialdata.py:185: UserWarning: The table is annotating 'comseg_boundaries', which 

In [9]:
sdata_main

SpatialData object, with associated Zarr store: /dss/dssfs03/pn52re/pn52re-dss-0001/cellseg-benchmark/samples/foxf2_s2_r1/sdata_z3_temp_SF.zarr
├── Points
│     └── 'foxf2_s2_r1_transcripts': DataFrame with shape: (<Delayed>, 9) (2D points)
├── Shapes
│     ├── 'boundaries_Baysor_2D_Cellpose_1_DAPI_PolyT_0.2': GeoDataFrame shape: (181678, 1) (2D shapes)
│     ├── 'boundaries_Baysor_2D_Cellpose_1_DAPI_PolyT_0.8': GeoDataFrame shape: (164856, 1) (2D shapes)
│     ├── 'boundaries_Baysor_2D_Cellpose_1_DAPI_Transcripts_0.2': GeoDataFrame shape: (174569, 1) (2D shapes)
│     ├── 'boundaries_Baysor_2D_Cellpose_1_DAPI_Transcripts_0.8': GeoDataFrame shape: (151323, 1) (2D shapes)
│     ├── 'boundaries_Baysor_2D_Cellpose_1_nuclei_model_1.0': GeoDataFrame shape: (180656, 1) (2D shapes)
│     ├── 'boundaries_Baysor_2D_Cellpose_2_DAPI_PolyT_0.2': GeoDataFrame shape: (174102, 1) (2D shapes)
│     ├── 'boundaries_Baysor_2D_Cellpose_2_DAPI_PolyT_0.8': GeoDataFrame shape: (150218, 1) (2D shapes)
│     

In [10]:
# default MERLIN output from Merscope (cyto2 on DAPI/PolyT)
sdata_utils.process_merlin_segmentation(
    sample_name, sample_paths, sdata_main, write_to_disk=True
)

Skipping Cellpose_1_Merlin as boundaries_Cellpose_1_Merlin exist already.
Skipping Cellpose_1_Merlin as adata_Cellpose_1_Merlin exist already.


In [10]:
# other segmentation approaches
seg_methods = os.listdir(os.path.join(data_dir, "samples", sample_name, "results"))
seg_methods

['Baysor_2D_Cellpose_2_DAPI_Transcripts_0.8',
 'ComSeg_Cellpose_1_DAPI_Transcripts',
 'Proseg_Cellpose_1_nuclei_model_vxl_2',
 'ComSeg',
 'Cellpose_1_nuclei_model',
 'Baysor_2D_Cellpose_2_DAPI_PolyT_0.2',
 'Baysor_2D_Cellpose_1_DAPI_Transcripts_0.8',
 'Proseg_Cellpose_1_DAPI_Transcripts_vxl_2',
 'Negative_Control_Rastered_5',
 'Cellpose_2_DAPI_PolyT',
 '.ipynb_checkpoints',
 'Proseg_Cellpose_2_DAPI_PolyT_vxl_2',
 'Proseg_pure',
 'metrics',
 'Proseg_Cellpose_1_DAPI_Transcripts',
 'Baysor_2D_Cellpose_1_DAPI_Transcripts_0.2',
 'Ficture',
 'Baysor_3D',
 'Baysor_2D_Cellpose_1_DAPI_PolyT_0.2',
 'Baysor_2D_Cellpose_2_DAPI_Transcripts_0.2',
 'Cellpose_1_DAPI_PolyT',
 'Proseg_Cellpose_2_DAPI_Transcripts_vxl_2',
 'Negative_Control_Rastered_10',
 'Negative_Control_Voronoi',
 'Proseg_Cellpose_1_DAPI_PolyT_vxl_2',
 'Proseg_Cellpose_2_DAPI_Transcripts',
 'Proseg_Cellpose_1_nuclei_model',
 'Cellpose_2_DAPI_Transcripts',
 'Cellpose_1_Merlin',
 'Negative_Control_Rastered_25',
 'Boms',
 'ComSeg_Cellpose

In [11]:
sdata_utils.integrate_segmentation_data(
    data_dir, sample_name, seg_methods, sdata_main, write_to_disk=True
);

Skipping boundary import of Baysor_2D_Cellpose_2_DAPI_Transcripts_0.8 as boundaries_Baysor_2D_Cellpose_2_DAPI_Transcripts_0.8 exist already.
Skipping adata import of Baysor_2D_Cellpose_2_DAPI_Transcripts_0.8 as adata_Baysor_2D_Cellpose_2_DAPI_Transcripts_0.8 exist already.
No boundaries/adata files found for ComSeg_Cellpose_1_DAPI_Transcripts. Skipping.
No boundaries/adata files found for Proseg_Cellpose_1_nuclei_model_vxl_2. Skipping.
Skipping boundary import of ComSeg as boundaries_ComSeg exist already.
Skipping adata import of ComSeg as adata_ComSeg exist already.
Skipping boundary import of Cellpose_1_nuclei_model as boundaries_Cellpose_1_nuclei_model exist already.
Skipping adata import of Cellpose_1_nuclei_model as adata_Cellpose_1_nuclei_model exist already.
Skipping boundary import of Baysor_2D_Cellpose_2_DAPI_PolyT_0.2 as boundaries_Baysor_2D_Cellpose_2_DAPI_PolyT_0.2 exist already.
Skipping adata import of Baysor_2D_Cellpose_2_DAPI_PolyT_0.2 as adata_Baysor_2D_Cellpose_2_DAPI

In [12]:
sdata_main.elements_paths_on_disk()

['points/foxf2_s2_r1_transcripts',
 'shapes/boundaries_Baysor_2D_Cellpose_1_DAPI_PolyT_0.2',
 'shapes/boundaries_Baysor_2D_Cellpose_1_DAPI_PolyT_0.8',
 'shapes/boundaries_Baysor_2D_Cellpose_1_DAPI_Transcripts_0.2',
 'shapes/boundaries_Baysor_2D_Cellpose_1_DAPI_Transcripts_0.8',
 'shapes/boundaries_Baysor_2D_Cellpose_1_nuclei_model_1.0',
 'shapes/boundaries_Baysor_2D_Cellpose_2_DAPI_PolyT_0.2',
 'shapes/boundaries_Baysor_2D_Cellpose_2_DAPI_PolyT_0.8',
 'shapes/boundaries_Baysor_2D_Cellpose_2_DAPI_Transcripts_0.2',
 'shapes/boundaries_Baysor_2D_Cellpose_2_DAPI_Transcripts_0.8',
 'shapes/boundaries_Baysor_3D',
 'shapes/boundaries_Cellpose_1_DAPI_PolyT',
 'shapes/boundaries_Cellpose_1_DAPI_Transcripts',
 'shapes/boundaries_Cellpose_1_nuclei_model',
 'shapes/boundaries_Cellpose_2_DAPI_PolyT',
 'shapes/boundaries_Cellpose_2_DAPI_Transcripts',
 'shapes/boundaries_ComSeg',
 'shapes/boundaries_Ficture',
 'shapes/boundaries_Negative_Control_Rastered_10',
 'shapes/boundaries_Negative_Control_Rast

In [13]:
for sample_name in sample_paths.keys():
    print("\nProcessing "+sample_name+"...")
    # load main sdata
    sdata_path = os.path.join(data_dir, "samples", sample_name, "sdata_z3_temp_SF.zarr")
    sdata_main = sd.read_zarr(sdata_path)
    seg_methods = os.listdir(os.path.join(data_dir, "samples", sample_name, "results"))
    sdata_utils.integrate_segmentation_data(
        data_dir, sample_name, seg_methods, sdata_main, write_to_disk=True
    );


Processing foxf2_s1_r0...


/dss/dsshome1/0C/ra98gaq/miniconda3/envs/sdata/lib/python3.11/site-packages/spatialdata/_core/spatialdata.py:185: UserWarning: The table is annotating 'baysor_boundaries', which is not present in the SpatialData object.
  self.validate_table_in_spatialdata(v)
/dss/dsshome1/0C/ra98gaq/miniconda3/envs/sdata/lib/python3.11/site-packages/spatialdata/_core/spatialdata.py:185: UserWarning: The table is annotating 'cellpose_boundaries', which is not present in the SpatialData object.
  self.validate_table_in_spatialdata(v)
/dss/dsshome1/0C/ra98gaq/miniconda3/envs/sdata/lib/python3.11/site-packages/spatialdata/_core/spatialdata.py:185: UserWarning: The table is annotating '20240229_Foxf2-Slide01-cp-WT-ECKO_region_0-ECKO000_polygons', which is not present in the SpatialData object.
  self.validate_table_in_spatialdata(v)
/dss/dsshome1/0C/ra98gaq/miniconda3/envs/sdata/lib/python3.11/site-packages/spatialdata/_core/spatialdata.py:185: UserWarning: The table is annotating 'image_patches', which is

Skipping boundary import of Baysor_2D_Cellpose_2_DAPI_Transcripts_0.8 as boundaries_Baysor_2D_Cellpose_2_DAPI_Transcripts_0.8 exist already.
Skipping adata import of Baysor_2D_Cellpose_2_DAPI_Transcripts_0.8 as adata_Baysor_2D_Cellpose_2_DAPI_Transcripts_0.8 exist already.
No boundaries/adata files found for ComSeg_Cellpose_1_DAPI_Transcripts. Skipping.
No boundaries/adata files found for Proseg_Cellpose_1_nuclei_model_vxl_2. Skipping.
Skipping boundary import of Cellpose_1_nuclei_model as boundaries_Cellpose_1_nuclei_model exist already.
Skipping adata import of Cellpose_1_nuclei_model as adata_Cellpose_1_nuclei_model exist already.
Skipping boundary import of Baysor_2D_Cellpose_2_DAPI_PolyT_0.2 as boundaries_Baysor_2D_Cellpose_2_DAPI_PolyT_0.2 exist already.
Skipping adata import of Baysor_2D_Cellpose_2_DAPI_PolyT_0.2 as adata_Baysor_2D_Cellpose_2_DAPI_PolyT_0.2 exist already.
Skipping boundary import of Baysor_2D_Cellpose_1_DAPI_Transcripts_0.8 as boundaries_Baysor_2D_Cellpose_1_DAP

/dss/dsshome1/0C/ra98gaq/miniconda3/envs/sdata/lib/python3.11/site-packages/spatialdata/_core/spatialdata.py:185: UserWarning: The table is annotating 'baysor_boundaries', which is not present in the SpatialData object.
  self.validate_table_in_spatialdata(v)
/dss/dsshome1/0C/ra98gaq/miniconda3/envs/sdata/lib/python3.11/site-packages/spatialdata/_core/spatialdata.py:185: UserWarning: The table is annotating 'cellpose_boundaries', which is not present in the SpatialData object.
  self.validate_table_in_spatialdata(v)
/dss/dsshome1/0C/ra98gaq/miniconda3/envs/sdata/lib/python3.11/site-packages/spatialdata/_core/spatialdata.py:185: UserWarning: The table is annotating '20240229_Foxf2-Slide01-cp-WT-ECKO_region_1-WT000_polygons', which is not present in the SpatialData object.
  self.validate_table_in_spatialdata(v)
/dss/dsshome1/0C/ra98gaq/miniconda3/envs/sdata/lib/python3.11/site-packages/spatialdata/_core/spatialdata.py:185: UserWarning: The table is annotating 'image_patches', which is n

Skipping boundary import of Baysor_2D_Cellpose_2_DAPI_Transcripts_0.8 as boundaries_Baysor_2D_Cellpose_2_DAPI_Transcripts_0.8 exist already.
Skipping adata import of Baysor_2D_Cellpose_2_DAPI_Transcripts_0.8 as adata_Baysor_2D_Cellpose_2_DAPI_Transcripts_0.8 exist already.
No boundaries/adata files found for ComSeg_Cellpose_1_DAPI_Transcripts. Skipping.
No boundaries/adata files found for Proseg_Cellpose_1_nuclei_model_vxl_2. Skipping.
Skipping boundary import of Cellpose_1_nuclei_model as boundaries_Cellpose_1_nuclei_model exist already.
Skipping adata import of Cellpose_1_nuclei_model as adata_Cellpose_1_nuclei_model exist already.
Skipping boundary import of Baysor_2D_Cellpose_2_DAPI_PolyT_0.2 as boundaries_Baysor_2D_Cellpose_2_DAPI_PolyT_0.2 exist already.
Skipping adata import of Baysor_2D_Cellpose_2_DAPI_PolyT_0.2 as adata_Baysor_2D_Cellpose_2_DAPI_PolyT_0.2 exist already.
Skipping boundary import of Baysor_2D_Cellpose_1_DAPI_Transcripts_0.8 as boundaries_Baysor_2D_Cellpose_1_DAP

...

Skipping boundary import of Baysor_2D_Cellpose_2_DAPI_Transcripts_0.8 as boundaries_Baysor_2D_Cellpose_2_DAPI_Transcripts_0.8 exist already.
Skipping adata import of Baysor_2D_Cellpose_2_DAPI_Transcripts_0.8 as adata_Baysor_2D_Cellpose_2_DAPI_Transcripts_0.8 exist already.
No boundaries/adata files found for ComSeg_Cellpose_1_DAPI_Transcripts. Skipping.
No boundaries/adata files found for Proseg_Cellpose_1_nuclei_model_vxl_2. Skipping.
Skipping boundary import of Cellpose_1_nuclei_model as boundaries_Cellpose_1_nuclei_model exist already.
Skipping adata import of Cellpose_1_nuclei_model as adata_Cellpose_1_nuclei_model exist already.
Skipping boundary import of Baysor_2D_Cellpose_2_DAPI_PolyT_0.2 as boundaries_Baysor_2D_Cellpose_2_DAPI_PolyT_0.2 exist already.
Skipping adata import of Baysor_2D_Cellpose_2_DAPI_PolyT_0.2 as adata_Baysor_2D_Cellpose_2_DAPI_PolyT_0.2 exist already.
Skipping boundary import of Baysor_2D_Cellpose_1_DAPI_Transcripts_0.8 as boundaries_Baysor_2D_Cellpose_1_DAP

/dss/dsshome1/0C/ra98gaq/miniconda3/envs/sdata/lib/python3.11/site-packages/spatialdata/_core/spatialdata.py:185: UserWarning: The table is annotating 'baysor_boundaries', which is not present in the SpatialData object.
  self.validate_table_in_spatialdata(v)
/dss/dsshome1/0C/ra98gaq/miniconda3/envs/sdata/lib/python3.11/site-packages/spatialdata/_core/spatialdata.py:185: UserWarning: The table is annotating 'cellpose_boundaries', which is not present in the SpatialData object.
  self.validate_table_in_spatialdata(v)
/dss/dsshome1/0C/ra98gaq/miniconda3/envs/sdata/lib/python3.11/site-packages/spatialdata/_core/spatialdata.py:185: UserWarning: The table is annotating '20240805_Foxf2-Slide07-cp-WT-GLKO_region_0-WT886_polygons', which is not present in the SpatialData object.
  self.validate_table_in_spatialdata(v)
/dss/dsshome1/0C/ra98gaq/miniconda3/envs/sdata/lib/python3.11/site-packages/spatialdata/_core/spatialdata.py:185: UserWarning: The table is annotating 'image_patches', which is n

Skipping boundary import of Baysor_2D_Cellpose_2_DAPI_Transcripts_0.8 as boundaries_Baysor_2D_Cellpose_2_DAPI_Transcripts_0.8 exist already.
Skipping adata import of Baysor_2D_Cellpose_2_DAPI_Transcripts_0.8 as adata_Baysor_2D_Cellpose_2_DAPI_Transcripts_0.8 exist already.
No boundaries/adata files found for ComSeg_Cellpose_1_DAPI_Transcripts. Skipping.
No boundaries/adata files found for Proseg_Cellpose_1_nuclei_model_vxl_2. Skipping.
Skipping boundary import of Cellpose_1_nuclei_model as boundaries_Cellpose_1_nuclei_model exist already.
Skipping adata import of Cellpose_1_nuclei_model as adata_Cellpose_1_nuclei_model exist already.
Skipping boundary import of Baysor_2D_Cellpose_2_DAPI_PolyT_0.2 as boundaries_Baysor_2D_Cellpose_2_DAPI_PolyT_0.2 exist already.
Skipping adata import of Baysor_2D_Cellpose_2_DAPI_PolyT_0.2 as adata_Baysor_2D_Cellpose_2_DAPI_PolyT_0.2 exist already.
Skipping boundary import of Baysor_2D_Cellpose_1_DAPI_Transcripts_0.8 as boundaries_Baysor_2D_Cellpose_1_DAP

/dss/dsshome1/0C/ra98gaq/miniconda3/envs/sdata/lib/python3.11/site-packages/spatialdata/_core/spatialdata.py:185: UserWarning: The table is annotating 'baysor_boundaries', which is not present in the SpatialData object.
  self.validate_table_in_spatialdata(v)
/dss/dsshome1/0C/ra98gaq/miniconda3/envs/sdata/lib/python3.11/site-packages/spatialdata/_core/spatialdata.py:185: UserWarning: The table is annotating 'cellpose_boundaries', which is not present in the SpatialData object.
  self.validate_table_in_spatialdata(v)
/dss/dsshome1/0C/ra98gaq/miniconda3/envs/sdata/lib/python3.11/site-packages/spatialdata/_core/spatialdata.py:185: UserWarning: The table is annotating '20240805_Foxf2-Slide07-cp-WT-GLKO_region_1-KO885_polygons', which is not present in the SpatialData object.
  self.validate_table_in_spatialdata(v)
/dss/dsshome1/0C/ra98gaq/miniconda3/envs/sdata/lib/python3.11/site-packages/spatialdata/_core/spatialdata.py:185: UserWarning: The table is annotating 'image_patches', which is n

Skipping boundary import of Baysor_2D_Cellpose_2_DAPI_Transcripts_0.8 as boundaries_Baysor_2D_Cellpose_2_DAPI_Transcripts_0.8 exist already.
Skipping adata import of Baysor_2D_Cellpose_2_DAPI_Transcripts_0.8 as adata_Baysor_2D_Cellpose_2_DAPI_Transcripts_0.8 exist already.
No boundaries/adata files found for ComSeg_Cellpose_1_DAPI_Transcripts. Skipping.
No boundaries/adata files found for Proseg_Cellpose_1_nuclei_model_vxl_2. Skipping.
Skipping boundary import of Cellpose_1_nuclei_model as boundaries_Cellpose_1_nuclei_model exist already.
Skipping adata import of Cellpose_1_nuclei_model as adata_Cellpose_1_nuclei_model exist already.
Skipping boundary import of Baysor_2D_Cellpose_2_DAPI_PolyT_0.2 as boundaries_Baysor_2D_Cellpose_2_DAPI_PolyT_0.2 exist already.
Skipping adata import of Baysor_2D_Cellpose_2_DAPI_PolyT_0.2 as adata_Baysor_2D_Cellpose_2_DAPI_PolyT_0.2 exist already.
Skipping boundary import of Baysor_2D_Cellpose_1_DAPI_Transcripts_0.8 as boundaries_Baysor_2D_Cellpose_1_DAP